# Fine-tuning a Transformer Model for Multilingual Named Entity Recognition using the CNER dataset (https://github.com/Babelscape/cner)

This guide will show you how to fine-tune 🤗 Transformers models for Concept Named Entity Recognition. You will use the 🤗
Datasets library to quickly load and preprocess the CNER dataset, getting them ready for training with PyTorch.

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
! pip install datasets transformers seqeval
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=e8123d2bdc0fb4702230638327f57e914ea93754d0400e68881be268cfd88036
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  

# Fine-tuning a model on a token classification task

Token classification refers to the task of classifying individual tokens in a sentence. In this notebook, you will learn how to fine-tune a model on the [CNER](https://github.com/Babelscape/cner) dataset to detect new entities.

## Loading the CNER dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.

The notebook should work with any token classification dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [ ]:
from datasets import load_dataset, load_metric, concatenate_datasets

datasets = load_dataset("Babelscape/cner")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

The CNER dataset has train, validation and test splits.

To access an actual element, you need to select a split first, then give an index:

In [ ]:
datasets["train"][0]

{'sentence_id': '3021080.0',
 'tokens': ['Erik',
  'Adolf',
  'von',
  'Willebrand',
  '(',
  '1',
  'February',
  '1870',
  '–',
  '12',
  'September',
  '1949',
  ')',
  'was',
  'a',
  'Finnish',
  'physician',
  'who',
  'made',
  'major',
  'contributions',
  'to',
  'hematology',
  '.'],
 'pos': ['PROPN',
  'PROPN',
  'PROPN',
  'PROPN',
  'PUNCT',
  'NUM',
  'PROPN',
  'NUM',
  'SYM',
  'NUM',
  'PROPN',
  'NUM',
  'PUNCT',
  'AUX',
  'DET',
  'ADJ',
  'NOUN',
  'PRON',
  'VERB',
  'ADJ',
  'NOUN',
  'ADP',
  'NOUN',
  'PUNCT'],
 'c_vs_ne': ['NE',
  'NE',
  'NE',
  'NE',
  'O',
  'C',
  'C',
  'C',
  'O',
  'C',
  'C',
  'C',
  'O',
  'O',
  'O',
  'NE',
  'C',
  'O',
  'O',
  'O',
  'C',
  'O',
  'C',
  'O'],
 'cner_tags': ['B-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'O',
  'B-DATETIME',
  'I-DATETIME',
  'I-DATETIME',
  'O',
  'B-DATETIME',
  'I-DATETIME',
  'I-DATETIME',
  'O',
  'O',
  'O',
  'B-GROUP',
  'B-PER',
  'O',
  'O',
  'O',
  'B-EVENT',
  'O',
  'B-DISCIPLINE',
  

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored the following:

In [ ]:
labels_vocab = {
    "O": 0,
    "B-ANIMAL": 1,
    "I-ANIMAL": 2,
    "B-DISEASE": 3,
    "I-DISEASE": 4,
    "B-DISCIPLINE": 5,
    "I-DISCIPLINE": 6,
    "B-LANGUAGE": 7,
    "I-LANGUAGE": 8,
    "B-EVENT": 9,
    "I-EVENT": 10,
    "B-FOOD": 11,
    "I-FOOD": 12,
    "B-ARTIFACT": 13,
    "I-ARTIFACT": 14,
    "B-MEDIA": 15,
    "I-MEDIA": 16,
    "B-GROUP": 17,
    "I-GROUP": 18,
    "B-ORG": 19,
    "I-ORG": 20,
    "B-PER": 21,
    "I-PER": 22,
    "B-STRUCT": 23,
    "I-STRUCT": 24,
    "B-LOC": 25,
    "I-LOC": 26,
    "B-PLANT": 27,
    "I-PLANT": 28,
    "B-MONEY": 29,
    "I-MONEY": 30,
    "B-BIOLOGY": 31,
    "I-BIOLOGY": 32,
    "B-MEASURE": 33,
    "I-MEASURE": 34,
    "B-SUPER": 35,
    "I-SUPER": 36,
    "B-CELESTIAL": 37,
    "I-CELESTIAL": 38,
    "B-LAW": 39,
    "I-LAW": 40,
    "B-SUBSTANCE": 41,
    "I-SUBSTANCE": 42,
    "B-PART": 43,
    "I-PART": 44,
    "B-CULTURE": 45,
    "I-CULTURE": 46,
    "B-PROPERTY": 47,
    "I-PROPERTY": 48,
    "B-FEELING": 49,
    "I-FEELING": 50,
    "B-PSYCH": 51,
    "I-PSYCH": 52,
    "B-RELATION": 53,
    "I-RELATION": 54,
    "B-DATETIME": 55,
    "I-DATETIME": 56,
    "B-ASSET": 57,
    "I-ASSET": 58
}

label_list = list(labels_vocab.keys())

labels_vocab_reverse = {v:k for k,v in labels_vocab.items()}

At this point, in order to obtain a multilingual model, we concatenate the training, validation and test sets of all languages.

If you want to obtain a monolingual model, you have to use a single language for each split.

In [ ]:
train_dataset = datasets["train"]

val_dataset = datasets["validation"]

test_dataset = datasets["test"]

We can see the training, validation and test sets all have a column for the tokens (the input texts split into words) and one column of labels for each kind of task we introduced before.

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
model_name = "bert-base-cased"
hf_model_name = "cner-model-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [ ]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 8667, 117, 1142, 1110, 1141, 5650, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [ ]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 8667, 117, 1142, 1110, 1141, 5650, 3325, 1154, 1734, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [ ]:
example = datasets["train"][0]
print(example["tokens"])

['Erik', 'Adolf', 'von', 'Willebrand', '(', '1', 'February', '1870', '–', '12', 'September', '1949', ')', 'was', 'a', 'Finnish', 'physician', 'who', 'made', 'major', 'contributions', 'to', 'hematology', '.']


In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'Erik', 'Adolf', 'von', 'Will', '##eb', '##rand', '(', '1', 'February', '1870', '–', '12', 'September', '1949', ')', 'was', 'a', 'Finnish', 'physician', 'who', 'made', 'major', 'contributions', 'to', 'hem', '##ato', '##logy', '.', '[SEP]']


Here the words "Ventana" and "condor" have been split in two subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 22, 22, 23, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["cner_tags_ids"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

30 30


Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [ ]:
label_all_tokens = False

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["cner_tags_ids"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 10129, 12670, 3262, 3100, 15581, 13141, 113, 122, 1428, 6339, 782, 1367, 1347, 3224, 114, 1108, 170, 6389, 7454, 1150, 1189, 1558, 5353, 1106, 23123, 10024, 6360, 119, 102], [101, 1124, 1145, 20429, 20443, 117, 184, 27655, 1105, 1301, 3818, 117, 1105, 1108, 1141, 1104, 1103, 1148, 6389, 15230, 1106, 1329, 26825, 1106, 7299, 170, 4267, 22377, 2941, 19737, 119, 102], [101, 1124, 1758, 1142, 8936, 1107, 4082, 117, 23902, 1122, 1121, 23123, 4184, 20473, 1465, 119, 102], [101, 1109, 2612, 1104, 1103, 3653, 1108, 1224, 2751, 1106, 1129, 170, 21344, 1104, 170, 4592, 117, 1208, 1227, 1112, 3262, 3100, 15581, 13141, 5318, 117, 1115, 13267, 23123, 15540, 14229, 119, 102], [101, 1230, 1266, 5609, 1106, 1103, 3619, 118, 3522, 7309, 1107, 5776, 119, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
train_tokenized = train_dataset.map(tokenize_and_align_labels, batched=True)
val_tokenized = val_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized = test_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/317590 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list), label2id=labels_vocab, id2label=labels_vocab_reverse)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
args = TrainingArguments(
    hf_model_name,
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=True,
    eval_steps=1000,
    save_steps=1000,
    metric_for_best_model = "eval_f1",
    load_best_model_at_end = True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate,customize the batch_size and the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [ ]:
metric = load_metric("seqeval")

<ipython-input-22-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


This metric takes list of labels for the predictions and references:

In [ ]:
labels = [list(label_list)[i] for i in example["cner_tags_ids"]]
metric.compute(predictions=[labels], references=[labels])

{'DATETIME': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'DISCIPLINE': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'EVENT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'GROUP': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
from transformers import EarlyStoppingCallback
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.add_callback(EarlyStoppingCallback(early_stopping_patience = 5))

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1000,0.549200,0.556555,0.591460,0.635529,0.612703,0.857178
2000,0.446300,0.539562,0.608204,0.647856,0.627404,0.861898
3000,0.406900,0.544578,0.606840,0.646487,0.626036,0.862998
4000,0.388200,0.531268,0.613572,0.656348,0.634240,0.864949
5000,0.366800,0.549354,0.624510,0.654705,0.639251,0.865659
6000,0.374200,0.561963,0.620636,0.657444,0.638510,0.864914
7000,0.351700,0.574019,0.615950,0.653746,0.634286,0.863282
8000,0.331100,0.579151,0.615395,0.650459,0.632441,0.863104
9000,0.327900,0.577958,0.621428,0.652239,0.636461,0.863920
10000,0.328700,0.589303,0.620075,0.653198,0.636206,0.864275


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=10000, training_loss=0.41018826141357423, metrics={'train_runtime': 1373.1541, 'train_samples_per_second': 231.285, 'train_steps_per_second': 28.911, 'total_flos': 2473382161878192.0, 'train_loss': 0.41018826141357423, 'epoch': 0.25})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [ ]:
predictions, labels, _ = trainer.predict(test_tokenized)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
model.push_to_hub(hf_model_name)


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from pprint import pprint
tokenizer = AutoTokenizer.from_pretrained("Babelscape/"+ hf_model_name)
model = AutoModelForTokenClassification.from_pretrained("Babelscape/"+ hf_model_name)

nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

ner_results = nlp("What is the seventh tallest mountain in North America?")
pprint(ner_results)
